In [2]:
#spark_session.stop()

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession ,Row
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

from pyspark.mllib.recommendation import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import explode

from operator import *
import os
import random

In [2]:
# initializing a spark session
spark_session = SparkSession.builder.appName('GRP10_MusicRec').getOrCreate()
# creating spark context for sql
SQL_context = SQLContext(spark_session.sparkContext)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/14 15:25:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [19]:
# reading csv data from local FS
# THIS PART NEEDS TO BE CHANGED TO HDFS VERSION
fPATH = 'train_triplets.txt'
triplets = spark_session.read.csv(fPATH, header = False, sep=r'\t')

# alter col name
triplets = triplets.withColumnRenamed('_c0','User').withColumnRenamed('_c1','Song').withColumnRenamed('_c2','Count')
triplets.show(n=10)
#print((triplets.count(), len(triplets.columns)))

+--------------------+------------------+-----+
|                User|              Song|Count|
+--------------------+------------------+-----+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|    1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|    1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|    2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|    1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|    1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|    1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|    2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|    1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|    1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|    1|
+--------------------+------------------+-----+
only showing top 10 rows



(48373586, 3)


In [23]:
# initializing a new dataframe
# generate hash for the upcoming processing
tripletsDF = triplets.withColumn('UserID', F.hash(col('User')))
tripletsDF = tripletsDF.withColumn('SongID', F.hash(col('Song')))
tripletsDF = tripletsDF.withColumn('CountNum', col('Count').cast(IntegerType()))
tripletsDF.show(10)
# This DF should have 48M entries, if we want to just have a test,
# limit it to 10k or 100k (still takes hours on single node)
#tripletsDF = tripletsDF.limit(10000)
# export a csv file for a glance view of tripletsDF
#tripletsDF.limit(200).write.csv('tripletsDF_preview')

+--------------------+------------------+-----+----------+-----------+--------+
|                User|              Song|Count|    UserID|     SongID|CountNum|
+--------------------+------------------+-----+----------+-----------+--------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|    1|1365117428| 1315780877|       1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|    1|1365117428|-1623759929|       1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|    2|1365117428|-1218290021|       2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|    1|1365117428|-1227648141|       1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|    1|1365117428| 2054460487|       1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|    1|1365117428| 1741539561|       1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|    2|1365117428| 1352462404|       2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|    1|1365117428|-1539040688|       1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|    1|1365117428| 1734414350|       1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0

In [32]:
fPATH2 = 'unique_tracks.txt'
# read another file with tracks and artist info
uTracks = spark_session.read.csv(fPATH2, sep = '<SEP>')
# renaming columns
uTracks = uTracks.withColumnRenamed("_c1","Song").withColumnRenamed("_c2","ArtistName").withColumnRenamed("_c3","Title")
# same as before
uTracks = uTracks.withColumn("SongID", F.hash(col("Song")))
uTracks = uTracks.drop(uTracks["_c0"]) # removing _c0 column
uTracks.show(10)

+------------------+--------------------+--------------------+-----------+
|              Song|          ArtistName|               Title|     SongID|
+------------------+--------------------+--------------------+-----------+
|SOQMMHC12AB0180CB8|    Faster Pussy cat|        Silent Night| -561391088|
|SOVFVAK12A8C1350D9|    Karkkiautomaatti|         Tanssi vaan| -147901473|
|SOGTUKN12AB017F4F1|      Hudson Mohawke|   No One Could Ever|  547521741|
|SOBNYVR12A8C13558C|         Yerba Brava|       Si Vos Querés| -624908617|
|SOHSBXH12A8C13B0DF|          Der Mystic|    Tangle Of Aspens| 1651792394|
|SOZVAPQ12A8C13B63C|    David Montgomery|Symphony No. 1 G ...| 1156129240|
|SOQVRHI12A6D4FB2D7|  Sasha / Turbulence|    We Have Got Love|-1236863542|
|SOEYRFT12AB018936C|          Kris Kross|   2 Da Beat Ch'yall| 1516150147|
|SOPMIYT12A6D4F851E|        Joseph Locke|             Goodbye|-1661794774|
|SOJCFMH12A8C13B0C2|The Sun Harbor's ...|Mama_ mama can't ...| -517389677|
+------------------+-----

In [37]:
# create datafram with matching songID from the dataframes tripletsDF and uTracks, and dropping duplicate column "Song"
matchingSongID = tripletsDF.join(uTracks,tripletsDF["SongID"] == uTracks["SongID"]).drop(uTracks["Song"])
matchingSongID.show(10)
print((matchingSongID.count(), len(matchingSongID.columns)))

+--------------------+------------------+-----+-----------+-----------+--------+--------------------+--------------------+-----------+
|                User|              Song|Count|     UserID|     SongID|CountNum|          ArtistName|               Title|     SongID|
+--------------------+------------------+-----+-----------+-----------+--------+--------------------+--------------------+-----------+
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|    2| 1365117428|-1218290021|       2|       Paco De Lucia|     Entre Dos Aguas|-1218290021|
|8937134734f869deb...|SOJCGJJ12A8AE48B5D|    5| -263623207| 1573068951|       5| The Pointer Sisters|  Jump (For My Love)| 1573068951|
|17aa9f6dbdf753831...|SONMWXV12AB01803B4|    1|-1748408205| -494221261|       1|      Jonas Brothers|           Year 3000| -494221261|
|d6589314c0a9bcbca...|SOSZNRJ12A8AE46E38|    6| 1468154379|-1903656676|       6|Michael Cera & El...| Anyone Else But You|-1903656676|
|5a905f000fc1ff3df...|SOAKDHD12A6310F1AE|    1| 1598577

(269015, 9)


In [17]:
# train test split
# make this seprately if we skip the cv process
(train, test) = tripletsDF.randomSplit([0.8,0.2], seed = 42)
# to choose if CV best result will be used, 0 for use our own hyperparameters, 1 for auto
CV = 0
# initializing ALS model
ALS_model = ALS(maxIter = 3, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')

In [6]:
# grid seraching for the best parameter, adding more costs exponential time
grid = ParamGridBuilder().addGrid(ALS_model.rank, [10]).addGrid(ALS_model.regParam, [0.01]).build()

# initialzie a RMSE evaluator
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'CountNum', predictionCol = 'prediction')

if CV == 1:
    # set up cross validation process
    CrossVal = CrossValidator(numFolds = 5, estimator = ALS_model, estimatorParamMaps = grid, evaluator = evaluator)

    model = CrossVal.fit(train)

    # get the best model from cross validation
    TopModel = model.bestModel

In [7]:
# using the best hyperparameters from cv process
if CV == 1:
    estimator1 = ALS(rank = TopModel._java_obj.parent().getRank(), regParam = TopModel._java_obj.parent().getRegParam(), maxIter = 10, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')
else:
    estimator1 = ALS(rank = 10, regParam = 0.01, maxIter = 3, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')
model1 = estimator1.fit(train)


# make predictions from this model and see the RMSE
predictions1 = model1.transform(test)
print('The RMSE is:', evaluator.evaluate(predictions1))

23/03/14 15:28:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/14 15:28:44 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/03/14 15:28:45 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


The RMSE is: 7.429359669657181


In [8]:
# generating recommendations (TOP 10)
Top10Rec = model1.recommendForAllUsers(10)
Top10Rec.printSchema()
#Top10Rec.show(30)

Top10RecExploded = Top10Rec.withColumn('rec', explode("recommendations")).select('UserID', col("rec.SongID"), col("rec.Rating"))
Top10RecExploded.show(30)

# export a glance view of Top10Recommendations
Top10RecExploded.limit(100).write.csv('Top10Recommendations')

/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


root
 |-- UserID: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- SongID: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



+-----------+-----------+---------+
|     UserID|     SongID|   Rating|
+-----------+-----------+---------+
|-2131509675| 1458280062|31.912842|
|-2131509675| 2055222246|20.631996|
|-2131509675|  879723940|16.991217|
|-2131509675| -935160721|12.000662|
|-2131509675| 1306018349|11.004046|
|-2131509675| -201959134|10.430031|
|-2131509675|-1204575976| 9.996399|
|-2131509675| -511952190| 9.994833|
|-2131509675|-1488449153| 9.918545|
|-2131509675|  119087386| 9.778154|
|-2110749797| 1458280062|  67.5327|
|-2110749797| 2055222246| 64.48921|
|-2110749797|-1977062199|24.984308|
|-2110749797| 1710454464|22.034925|
|-2110749797| -787533004|20.331532|
|-2110749797| 2140071289|16.740631|
|-2110749797| -450919340|16.572977|
|-2110749797| -890345102| 16.36375|
|-2110749797|  249097782|15.978709|
|-2110749797|  535662807|13.997083|
|-2086707023| -935160721| 27.60996|
|-2086707023|-1297915589|21.672539|
|-2086707023|  734367063|14.257734|
|-2086707023| 2140071289|13.756442|
|-2086707023|  336855604|13.

In [9]:
# export a glance view of Top10Recommendations
Top10RecExploded.write.csv('Top10Recommendations_50000')

In [11]:
# join two dataframes so that we can match song title with id
preview1 = Top10RecExploded.join(uTracks, on = 'SongID').sort('Rating', ascending = False)

# if nothing shows here, you need to use a larger limit for tripletsDF, e.g. 100,000
preview1.show(10)

+-----------+-----------+----------+------------------+------------------+------------+-----------------+
|     SongID|     UserID|    Rating|               _c0|              Song|  ArtistName|            Title|
+-----------+-----------+----------+------------------+------------------+------------+-----------------+
|-1687914835| -585083843| 18.760351|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835| -100314973| 18.429499|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835|  -35879908| 17.881044|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835|  588653661|  17.53953|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835|  549596506| 15.353599|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835|-2007952265| 14.905753|TRMHHRK128F932A818|SOLPVAQ12AB017EB35|   Nick Lowe|All Men Are Liars|
|-1687914835| 1020076396|14.6285305|TRMHHRK128

In [12]:
AVGtripletDF = tripletsDF.groupby('Song').avg('CountNum')
AVGtripletDF = tripletsDF.join(AVGtripletDF,on = 'Song')
AVGtripletDF = AVGtripletDF.withColumn('Rating',F.col('CountNum') / F.col('avg(CountNum)'))
AVGtripletDF.limit(10).show()

+------------------+--------------------+-----+----------+-----------+--------+-------------+-------------------+
|              Song|                User|Count|    UserID|     SongID|CountNum|avg(CountNum)|             Rating|
+------------------+--------------------+-----+----------+-----------+--------+-------------+-------------------+
|SOAKIMP12A8C130995|b80344d063b5ccb32...|    1|1365117428| 1315780877|       1|          1.0|                1.0|
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|    1|1365117428|-1623759929|       1|          1.0|                1.0|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|    2|1365117428|-1218290021|       2|          1.5| 1.3333333333333333|
|SOBFNSP12AF72A0E22|b80344d063b5ccb32...|    1|1365117428|-1227648141|       1|          1.0|                1.0|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|    1|1365117428| 2054460487|       1|          1.0|                1.0|
|SOBNZDC12A6D4FC103|b80344d063b5ccb32...|    1|1365117428| 1741539561|       1|         

In [13]:
# train test split
# make this seprately if we skip the cv process
(train, test) = AVGtripletDF.randomSplit([0.8,0.2], seed = 42)
# to choose if CV best result will be used, 0 for use our own hyperparameters, 1 for auto
CV = 0
# initializing ALS model
ALS_model = ALS(maxIter = 3, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')


# grid seraching for the best parameter, adding more costs exponential time
grid = ParamGridBuilder().addGrid(ALS_model.rank, [10]).addGrid(ALS_model.regParam, [0.01]).build()

# initialzie a RMSE evaluator
evaluator = RegressionEvaluator(metricName = 'rmse', labelCol = 'CountNum', predictionCol = 'prediction')

if CV == 1:
    # set up cross validation process
    CrossVal = CrossValidator(numFolds = 5, estimator = ALS_model, estimatorParamMaps = grid, evaluator = evaluator)

    model = CrossVal.fit(train)

    # get the best model from cross validation
    TopModel = model.bestModel
    

# using the best hyperparameters from cv process
if CV == 1:
    estimator2 = ALS(rank = TopModel._java_obj.parent().getRank(), regParam = TopModel._java_obj.parent().getRegParam(), maxIter = 10, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')
else:
    estimator2 = ALS(rank = 10, regParam = 0.01, maxIter = 3, userCol = "UserID", itemCol = 'SongID', ratingCol = 'CountNum', coldStartStrategy = 'drop')
model2 = estimator2.fit(train)


# make predictions from this model and see the RMSE
predictions2 = model2.transform(test)
print('The RMSE is:', evaluator.evaluate(predictions2))

The RMSE is: 7.407911345459607


In [14]:
Top10NewRec = model2.recommendForAllUsers(10)
Top10NewRecExploded = Top10NewRec.withColumn('rec', explode("recommendations")).select('UserID', col("rec.SongID"), col("rec.Rating"))
Top10NewRecExploded.show(20)

+-----------+-----------+---------+
|     UserID|     SongID|   Rating|
+-----------+-----------+---------+
|-2131509675|-1930412723|23.561878|
|-2131509675|  879723940|16.996307|
|-2131509675| -474679864|12.047587|
|-2131509675| -201959134|11.697766|
|-2131509675|  652118490|11.000159|
|-2131509675| 1306018349| 10.97734|
|-2131509675|  119087386|10.966654|
|-2131509675| -450919340|10.756085|
|-2131509675| -511952190| 9.997828|
|-2131509675| 1350330894| 9.745574|
|-2110749797|-1930412723| 81.26264|
|-2110749797|-1297511273|28.665258|
|-2110749797|-1977062199|24.993374|
|-2110749797| 1710454464|21.972082|
|-2110749797| 1689226843|19.066015|
|-2110749797| -787533004|18.352938|
|-2110749797| -890345102|17.714481|
|-2110749797|  249097782|15.965396|
|-2110749797| -219465134|13.818289|
|-2110749797| 1798603311| 9.715985|
+-----------+-----------+---------+
only showing top 20 rows



In [15]:
preview2 = Top10NewRecExploded.join(uTracks, on='SongID').filter('UserID = -1935480550').sort('Rating',ascending=False).select('SongID','UserID','Rating','Title')
preview2.show()

+------+------+------+-----+
|SongID|UserID|Rating|Title|
+------+------+------+-----+
+------+------+------+-----+



In [16]:
preview3 = AVGtripletDF.join(uTracks,on="SongID").filter('UserID = -1935480550').sort('CountNum', ascending=False).select('SongID','UserID','CountNum','Title')
preview3.show()

+------+------+--------+-----+
|SongID|UserID|CountNum|Title|
+------+------+--------+-----+
+------+------+--------+-----+



In [25]:
preview2.write.csv('Top10NewRecommendations_All')
preview3.write.csv('AVGtripletDF_All')